In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/world-foodfeed-production/FAO.csv


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py 
import plotly.graph_objs as go
import scipy.cluster.hierarchy as shc
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation
from sklearn.preprocessing import StandardScaler
import os

df = pd.read_csv("../input/world-foodfeed-production/FAO.csv",  encoding = "ISO-8859-1")
df.head()

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,...,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,...,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,...,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,...,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,...,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


DATA DESCRIPTION
Each row of this dataset contains the amount (values represent 1000 tonnes) of Feed/Food produced by each country ( 'Area' ) from 1961 to 2013 for a particular Item.

More metadata are included such as Area Abbreviation, Area/Item/Element Code, latitude, longitude, not used in this analysis.

The dataset is reduced containg only columns: Area, Item, Element, Y1961-Y2013

In [3]:
df.shape

(21477, 63)

ADDITION OF POPULATION AND SURFACE DIMENSIONS
In order to make analysis richer I've decided to add to this dataset information about the population and the surface area of each country.

Data is taken again from FAO stats website, considering the year of 2013.

Population data is specify as Million of people. Surface area instead as 1000 acres.

Datasets are merged with existing dataset considering 'Area' as key.

In [8]:
#In order to not have problems of consistency:
df['Area'].replace(['Swaziland'], 'Eswatini', inplace=True)
df['Area'].replace(['The former Yugoslav Republic of Macedonia'], 'North Macedonia', inplace=True)

#GET NEW DATA
df_pop = pd.read_csv("/kaggle/input/world-population/FAOSTAT_data_6-13-2019.csv")
df_area = pd.read_csv("/kaggle/input/countries-area-2013/countries_area_2013.csv")
df_pop = pd.DataFrame({'Area': df_pop['Area'] , 'Population': df_pop['Value'] })
df_area = pd.DataFrame({'Area' : df_area['Area'], 'Surface': df_area['Value']})
#add missing line
df_area = df_area.append({'Area' : 'Sudan' , 'Surface' : 1886} , ignore_index=True)

#MERGE OF TABLES
d1 = pd.DataFrame(df.loc[:, ['Area', 'Item', 'Element']])
data = pd.merge(d1, df_pop, on='Area', how='left')
new_data = pd.merge(data, df_area, on='Area', how='left')

d2 = df.loc[:, 'Y1961':'Y2013']
data = new_data.join(d2)
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/FAOSTAT_data_6-13-2019.csv'